# Especialização em Ciência de Dados - PUC-Rio
# Machine Learning
## Ensembles e Seleção de Atributos

# Importação dos dados
Usaremos a mesma base de dados que trabalhamos anteriormente, o dataset Pima Indians Diabetes. O trecho de código a seguir é similar ao que fizemos em laboratórios anteriores (exceto pela padronização dos dados) e serve apenas para importar os dados e separá-los em entradas (X) e saídas (Y).

In [0]:
# Carrega arquivo csv usando Pandas usando uma URL

# Importa todo o pacote Pandas
import pandas as pd

# Importa a função datasets
from sklearn import datasets

# Importa a função scale
from sklearn.preprocessing import scale

# Informa a URL de importação do dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"

# Informa o cabeçalho das colunas
colunas = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']

# Lê o arquivo utilizando as colunas informadas
dataset = pd.read_csv(url, names=colunas, skiprows=0, delimiter=',')

# Pega apenas os dados do dataset e guardando em um array
array = dataset.values

# Separa o array em variáveis preditoras (X) e variável target (Y)
# Coloca os dados na mesma escala (padronização)
X = scale(array[:,0:8])
Y = array[:,8]

In [0]:
# Imports
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

seed = 7
kfold = KFold(n_splits=10, random_state=seed)

# Ensembles

## Bagging

### Bagging com Árvores de Decisão
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html

Vamos usar o classificador BaggingClassifier com modelos CART, criando um total de 100 árvores.

In [0]:
# Imports
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

cart = DecisionTreeClassifier()
num_trees = 100
modelo = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed) 
results = cross_val_score(modelo, X, Y, cv=kfold)
print(results.mean())

0.7694463431305537


Também podemos usar outro modelo com o Bagging, como o KNN.

In [0]:
# Imports
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

modelo = BaggingClassifier(base_estimator=KNeighborsClassifier(), random_state=seed) 
results = cross_val_score(modelo, X, Y, cv=kfold)
print(results.mean())

0.7551777170198223


### Random Forest e Extra Tree Classifier
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html

Iremos primeiro construir um modelo CART simples para comparação e, em seguida, construir um modelo RandomForest para usando a classe RandomForestClassifier e um modelo ExtraTrees usando a classe ExtraTreesClassifier.

Para ambos os modelos ensemble, usaremos 100 árvores e pontos de divisão escolhidos com uma seleção randômica de 3 atributos.

In [0]:
# Imports
from sklearn.tree import DecisionTreeClassifier

modelo = DecisionTreeClassifier() 
results = cross_val_score(modelo, X, Y, cv=kfold)
print(results.mean())

0.6965481886534518


In [0]:
num_trees = 100
max_features = 3

In [0]:
# Imports
from sklearn.ensemble import RandomForestClassifier

modelo = RandomForestClassifier(n_estimators=num_trees, max_features=max_features, random_state=seed) 
results = cross_val_score(modelo, X, Y, cv=kfold)
print(results.mean())

0.7720779220779221


In [0]:
# Imports
from sklearn.ensemble import ExtraTreesClassifier

modelo = ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features, random_state=seed) 
results = cross_val_score(modelo, X, Y, cv=kfold)
print(results.mean())

0.7603725222146275


## Boosting

### Adaboost
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html

Iremos criar um modelo AdaBoost para classificação usando a classe AdaBoostClassifier. Serão construídas 100 árvores de decisão em sequência usando o algoritmo AdaBoost. O AdaBoost pode ser utilizado com vários modelos base, mas quando o parâmetro base_estimator não é passado, ele usa o DecisionTreeClassifier como padrão.

In [0]:
# Import
from sklearn.ensemble import AdaBoostClassifier

modelo = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
results = cross_val_score(modelo, X, Y, cv=kfold)
print(results.mean())

0.7408407382091593


### Stochastic Gradient Boosting
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

Iremos criar um modelo Gradient Boosting para classificação usando a classe GradientBoostingClassifier. Serão construídas 100 árvores de decisão em sequência usando o algoritmo Gradient Boosting.

In [0]:
# Imports
from sklearn.ensemble import GradientBoostingClassifier

modelo = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed) 
results = cross_val_score(modelo, X, Y, cv=kfold)
print(results.mean()) 

0.7669002050580999


## Voting Ensemble

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html

Iremos criar um ensemble de votação para classificação usando a classe VotingClassifier. Serão combinadas as predições de regressão logística, CART e SVM para o mesmo problema.

In [0]:
# Imports 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

# create the sub models
estimators = []
modelo1 = LogisticRegression(solver='liblinear')
estimators.append(('logistic', modelo1))
modelo2 = DecisionTreeClassifier()
estimators.append(('cart', modelo2))
modelo3 = SVC(gamma='auto')
estimators.append(('svm', modelo3))

# create the ensemble model
ensemble = VotingClassifier(estimators)
results = cross_val_score(ensemble, X, Y, cv=kfold)
print(results.mean())

0.7733937115516063


# Seleção de Atributos

## Seleção Univariada
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html

A função **SelectKBest()** pode ser usada com diversos testes estatísticos, para selecionar os atributos. Vamos usar o teste qui-quadrado e selecionar os 4 melhores atributos que podem ser usados como variáveis preditoras.

> O teste estatístico qui-quadrado é aplicado a dados categóricos para avaliar o quão provável é que qualquer diferença absoluta observada aconteça ao **acaso**, verificando se a frequência absoluta **observada** de uma variável é significativamente diferente da distribuição de frequência absoluta **esperada**. *Para saber mais: https://pt.wikipedia.org/wiki/Teste_qui-quadrado_de_Pearson*

In [0]:
# Imports
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from numpy import set_printoptions

# Função para seleção de atributos
best_var = SelectKBest(score_func = chi2, k = 4)

# Executa a função de pontuação em (X, Y) e obtém os atributos selecionados
fit = best_var.fit(abs(X), Y)

# Reduz X para os atributos selecionados
features = fit.transform(X)

# Resultados
print('\nNúmero original de atributos:', X.shape[1])
print('\nNúmero reduzido de atributos:', features.shape[1])

# Exibe os atributos orginais
print("\nAtributos Originais:", dataset.columns[0:8])

# Exibe as pontuações de cada atributos e os 4 escolhidas (com as pontuações mais altas): plas, teste, massa e idade.
# (Basta mapear manualmente o índice dos nomes dos respectivos atributos)
set_printoptions(precision=3)
print(fit.scores_)


Número original de atributos: 8

Número reduzido de atributos: 4

Atributos Originais: Index(['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age'], dtype='object')
[ 6.605 11.619  3.63   7.452 17.004  0.572  3.687  0.478]


## Eliminação Recursiva de Atributos
https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html

Iremos aplicar a técnica de eliminação recursiva de atributos com um algoritmo de Regressão Logística para selecionar as 3 melhores variáveis preditoras.

In [0]:
# Imports
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# Criação do modelo
modelo = LogisticRegression(solver='liblinear')

# Eliminação Recursiva de Variáveis
rfe = RFE(modelo, 3)
fit = rfe.fit(X, Y)

# Print dos resultados
print("Atributos preditores:", dataset.columns[0:8])
# Exibe os atributos selecionados (marcados como True em "Atributos Selecionados" e com valor 1 em "Ranking dos Atributos"): preg, mass e pedi.
# (Basta mapear manualmente o índice dos nomes dos respectivos atributos)
print("\nAtributos selecionados: %s" % fit.support_)
print("\nRanking de atributos: %s" % fit.ranking_)
print("\nQtd de melhores Atributos: %d" % fit.n_features_)

Atributos preditores: Index(['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age'], dtype='object')

Atributos selecionados: [ True  True False False False  True False False]

Ranking de atributos: [1 1 3 6 5 1 2 4]

Qtd de melhores Atributos: 3


## Análise de Componentes Principais (PCA)
http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

Iremos aplicar a técnica de Análise de Componentes Principais selecionando 3 componentes principais.

In [0]:
# Imports
from sklearn.decomposition import PCA

# Função PCA
pca = PCA(n_components=3)
fit = pca.fit(X)

# Resumo dos componentes - tem pouca semelhança com os dados de origem
print("Porcentagem de variância explicada por cada um dos componentes: %s" % fit.explained_variance_ratio_)
print("\n", fit.components_)

Porcentagem de variância explicada por cada um dos componentes: [0.262 0.216 0.129]

 [[ 0.128  0.393  0.36   0.44   0.435  0.452  0.271  0.198]
 [ 0.594  0.174  0.184 -0.332 -0.251 -0.101 -0.122  0.621]
 [-0.013  0.468 -0.535 -0.238  0.337 -0.362  0.433  0.075]]


## Importância de Atributos
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html

Iremos construir um classificador ExtraTreesClassifier.

In [0]:
# Imports
from sklearn.ensemble import ExtraTreesClassifier

# Criação do modelo para seleção de atributos
modelo = ExtraTreesClassifier(n_estimators=100)
modelo.fit(X, Y)

# Exibe a pontuação de importância para cada atributo (quanto maior a pontuação, mais importante é o atributo). Atributos selecionados: plas, idade e massa.
print(modelo.feature_importances_)

[0.112 0.231 0.099 0.08  0.077 0.138 0.122 0.142]
